<a href="https://colab.research.google.com/github/Anshuman-02905/Custom_Satellite_Scripts/blob/main/DATA_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [3]:
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np

In [4]:
state_path="/content/drive/MyDrive/States/State_collection.csv"
df=pd.read_csv(state_path)

In [5]:
df.head()

,gaul_adm2,adm2_name,str2_year,exp2_year,status,disp_area
0,40621,Ryansoro,1000,3000,Member State,NO
1,40667,Ndava,1000,3000,Member State,NO
2,40593,Buyengero,1000,3000,Member State,NO
3,40580,Bugarama,2004,3000,Member State,NO
4,40591,Burambi,1000,3000,Member State,NO


In [6]:
Map=geemap.Map() 

In [21]:
Map=geemap.Map()
state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Ryansoro'))
sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
  }
#Map.addLayer(sentinel.mean().divide(10000).clip(state), visualization, 'RGB')

urban = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

visualization = {
  'min': 0,
  'max': 255,
}
Map.addLayer(urban.clip(state), visualization, "Human1")
urban=urban.expression('b("settlement") == 255 ? 255:0',{'settlement':urban.select('settlement')})
Map.addLayer(urban.clip(state), visualization, "Human2")
Map.centerObject(state)

geemap.ee_export_image(urban, filename="final.tif", scale=50, region=state.geometry(), file_per_band=False)
geemap.ee_export_image(image, filename="sentinel.tif", scale=50, region=state.geometry(), file_per_band=False)

options_list = ['-ot Byte','-of PNG'] 
options_string = " ".join(options_list)
gdal.Translate("final.png", "final.tif", options=options_string) 

options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
options_string = " ".join(options_list) 
gdal.Translate("Sentinel.png","sentinel.tif", options=options_string)




Generating URL ...
Please wait ...
Data downloaded to /content/final.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel.tif


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f6dfaa1d1b0> >

In [14]:
Map

Map(center=[-3.722100847072025, 29.797068808681892], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:

!pip install retry
import ee
import logging
import multiprocessing
import requests
import shutil
from retry import retry


"""
This tool demonstrates extracting data from Earth Engine using parallel
request and getThumbURL.
"""

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

def getRequests():
  """Generates a list of work items to be downloaded.
  Produces 1000 random points in each of the RESOLVE ecoregions in the ROI.
  """
  # Get our ROI from the GAUL regions
  gaul = ee.FeatureCollection('FAO/GAUL/2015/level1')
  roi = gaul.filter('ADM1_NAME == "Colorado"').first().geometry()

  # To stratify by RESOLVE ecoregion, paint the ECO_IDs into an image.
  resolve = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
  ecoregions = (resolve.reduceToImage(['ECO_ID'], ee.Reducer.first())
                .clip(roi).rename('ECO_ID'))
  points = ecoregions.stratifiedSample(
      numPoints=1000,
      classBand='ECO_ID',
      region=roi,
      scale=100,
      geometries=True)

  return points.aggregate_array('.geo').getInfo()

'''
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
  """Handle the HTTP requests to download an image."""

  # Generate the desired image from the given point.
  point = ee.Geometry.Point(point['coordinates'])
  region = point.buffer(127).bounds()
  image = (ee.ImageCollection('USDA/NAIP/DOQQ')
           .filterBounds(region)
           .filterDate('2019', '2020')
           .mosaic()
           .clip(region)
           .select('R', 'G', 'B'))

  # Fetch the URL from which to download the image.
  url = image.getThumbURL({
      'region': region,
      'dimensions': '256x256',
      'format': 'png'})

  # Handle downloading the actual pixels.
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    r.raise_for_status()

  filename = 'tile_%05d.png' % index
  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
  print("Done: ", index)


if __name__ == '__main__':
  logging.basicConfig()
  items = getRequests()

  pool = multiprocessing.Pool(25)
  pool.starmap(getResult, enumerate(items))

  pool.close()'''

Done:  413
Done:  651
Done:  930
Done:  453
Done:  254
Done:  572
Done:  294
Done:  891
Done:  812
Done:  333
Done:  172
Done:  14
Done:  732
Done:  614
Done:  532
Done:  214
Done:  852
Done:  972
Done:  692
Done:  255
Done:  454
Done:  414
Done:  652
Done:  773
Done:  892
Done:  334
Done:  931
Done:  372
Done:  295
Done:  813
Done:  15
Done:  573
Done:  853
Done:  173
Done:  615
Done:  733
Done:  693
Done:  973
Done:  90
Done:  54
Done:  215
Done:  415
Done:  533
Done:  16
Done:  774
Done:  854
Done:  455
Done:  932
Done:  174
Done:  256
Done:  373
Done:  893
Done:  574
Done:  616
Done:  734
Done:  814


Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-6:
Process ForkPoolWorker-21:
Process ForkPoolWorker-17:
Process ForkPoolWorker-23:
Process ForkPoolWorker-7:
Process ForkPoolWorker-14:
Process ForkPoolWorker-19:
Process ForkPoolWorker-25:
Process ForkPoolWorker-9:
Process ForkPoolWorker-2:
Process ForkPoolWorker-18:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py"

KeyboardInterrupt: ignored


During handling of the above exception, another exception occurred:


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)

During handling of the above exception, another exception occurred:

TypeError: getresponse() got an unexpected keyword argument 'buffering'
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = con